# BDSN Assignment

### A21025

#### Hyundai Heavy Industries is one of the world's largest ship manufacturing companies and builds cruise liners.

#### They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

We will Import all requireed Libraries.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import corr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 45.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=052b3b5754f149123a3e2a45679795c9ccb3808aa4a1b2d9c617c1f59463b1d7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()



In [ ]:
spark

In [ ]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

Upload the Dataset

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/ships.csv',inferSchema=True,header=True)

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [ ]:
df.show()

+---+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|_c0|  Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+---+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|  0|    Journey|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|  1|      Quest|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|  2|Celebration|   Carnival| 26| 47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|  3|   Conquest|   Carnival| 11|  110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|  4|    Destiny|   Carnival| 17|101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|  5|    Ecstasy|   Carnival| 22| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|  6|    Elation|   Carnival| 15| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|  7|    Fantasy|   Carnival| 23| 70.367|     20.56|  8.55| 10.22|            34.23| 9.2|
|  8|Fasci

In [ ]:
df.describe().show()

+-------+------------------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|               _c0|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+------------------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|               158|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean|              78.5| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|45.754781170933384|     null|       null| 7.615691058751413|37.229540025907866|9.6770947751

#### Ship Name is a useless arbitrary string, but the cruise_line itself may be useful. Let's make it into a categorical variable.



In [ ]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



We will convert curise line into cruise categorical valuse using StringIndexer.

In [ ]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.head(5)

[Row(_c0=0, Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(_c0=1, Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(_c0=2, Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(_c0=3, Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(_c0=4, Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)]

In [ ]:
indexed.columns

['_c0',
 'Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

We will convert all the features into vectors using VectorAssembler.

In [ ]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.277,6.94,...|3.55|
|[6.0,30.277,6.94,...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.239,37.0...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [ ]:
final_data = output.select("features", "crew")

Split the Dataset into Train and Test (70 - 30)

In [ ]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

Create a Linear Regression Model object

In [ ]:
lr = LinearRegression(labelCol='crew')

Fit the model to the data and call this model lrModel

In [ ]:
lrModel = lr.fit(train_data)

Print the coefficients and intercept for linear regression

In [ ]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.006003614763547642,0.005816222140279991,-0.1340582665202412,0.4713718698405512,0.8544279060933411,-0.005989971174231681,0.042526741363031945] Intercept: -1.3009066092088724


We'll evaluate the Model.

In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 0.8359817428705837
MSE: 0.6988654744129387
R2: 0.9277491710687162


In [ ]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [ ]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

